# 第7章 ベイジアンネットワークの実装

#### ・本章では、ベイジアンネットワークを活用した因果探索について解説、実装する .

## 7-1 ベイジアンネットワークとは

#### ・第3章で解説したグラフ表現(因果ダイアグラム)によるネットワーク図をベースとした、変数間の関係性を表す手法 .

#### そのため、有向(Directed)で非循環(Acyclic)なグラフであるDAG(Directed Acyclic Graph)の概念、及び因果推論においてどの変数を考慮すべきかを判断するd分離の概念は、ベイジアンネットワークでも共通である .

#### 非循環ではないネットワークを扱う手法もあるが、今回はDAGである因果ダイアグラムを対象とする .

## スケルトンとPDAG(Partially DAG)

#### ・スケルトンとは骨格構造のことであり、DAGにおいて、因果の方向を示す矢印がないグラフを示す .

#### スケルトンは、最終的なDAGにおいてどのノードとどのノードの間に繋がり(エッジ)が存在するのかの情報を与えてくれる .

#### ・PDAGはスケルトンとDAGの中間にあたる存在である .　一部の辺(エッジ)には因果の方向を示す矢印があるが、その他のエッジは無向となっているグラフを指す .

![alt text](pict1.png)

## ベイジアンネットワークのノード間の関係性

#### ・第6章のLiNGAMにおいては、構造方程式モデルを用意して、数式を利用して変数間の関係性を表現した .

#### ・ベイジアンネットワークでは構造方程式を用いず、因果関係のあるノード間の関係性を、"条件付き確率表(CPT : Conditional Probabilities Tables)と呼ばれる手法を利用する .

#### ・前出の図7.1.1における、変数$x_1$,$x_2$,$x_3$のCPTを例に示す(図7.1.2).　ここで変数$x_1$,$x_2$,$x_3$はすべて0か1となる2値変数とする .

![alt text](pict2.png)

#### ・図7.1.2では、変数$x_1$は確率60%で0、40%で1の値となる .　変数$x_2$は確率40%で0、60%で1の値となる .

#### 変数$x_3$は変数$x_1$、変数$x_2$から因果の矢印が伸びており、因果関係にあるため、変数$x_1$と変数$x_2$の値に応じて値が変わる .

#### 変数$x_3$がとる値の確率は$P(x_3=0|x_1,x_2)$で示される .　$x_3$のみが記載されているが、$x_3$の値は0か1なので、0のときの確率を1.0から引き算したものが$x_3 = 1$の確率となる .

#### そのため、$P(x_3=1|x_1,x_2)$の表は、図7.1.2では掲載を省略している .

#### ・例えば、$x_1=0$、$x_2=1$の場合、$x_3$は30%の確率で0、70%の確率で1になるということを示している .

#### ・ベイジアンネットワークにおいては変数間の関係を条件付き確率表CPTで表せるよう、変数を離散値として扱うことが一般的である .

#### 連続値として扱う手法も存在しますが、本書では変数を離散値に限る .

#### 仮に変数が連続値である場合には、ビンで区切ることで離散値に変換し、離散変数として扱う(連続変数の離散化については7.5節で扱う) .

## 7-2 ネットワークの当てはまりの良さを測る方法

#### ・ベイジアンネットワークにおいて、データに対するネットワークの当てはまりの良さを示す指標は複数存在する .

#### AIC(Akaike information criterion)
#### BIC(Bayesian information criterion)
#### MDL符号(minimum description length)
#### BDe(Bayesian Dirichlet equivalent)
#### BDeu(Bayesian Dirichlet equivalent uniform)
#### K2

#### ・本書ではBICを取り上げ、その概要を解説する .　BICは以下の式で計算される .

$$
BIC_m = -2\, l_m(\theta_m \mid X) + k_m (\log N)
$$

#### ・ここで下付き文字のmはモデルを表す .　すなわち、現在対象としているベイジアンネットワークのDAGです .　$X$は実測されたデータを示す .

#### $k_m$はモデルのパラメータ数を、$N$は実測されたデータ数を示す .　$l_m(\theta_m \mid X)$はデータ$X$のもとでのモデルの対数尤度である .

#### そして、$\theta_m $がベイジアンネットワークの構造、すなわち条件付き確率表CPTを示す .

![alt text](pict3.png)

#### ・図7.2.1の左側の図がモデルmを示す .　モデルのパラメータ数$k_m$はCPT(条件付き確率表)の行数となり6である .　

#### 図7.2.1ではCPTは8行ありますが、すべて2値変数なので、0の確率$P(0)$が分かれば、1の確率は"$1.0-P(0)$"である .

#### そのため、変数$x_1$と変数$x_2$の表はパラメータ数としては1行より、パラメータ数は6となる .

### プログラム実行前の設定

In [1]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [2]:
# 使用するパッケージ（ライブラリと関数）を定義
from numpy.random import *
import pandas as pd

#### ・次に、図7.2.1のベイジアンネットワークから、例えば以下の表7.2.1に示す10個のデータ$X$が観測されたとする .

![alt text](pict4.png)

#### 表7.2.1のデータ$X$を生成したコードは以下のようになる .

In [3]:
# データ数
num_data = 10

# x1：0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2：0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df = pd.DataFrame({'x1': x1,
                   'x2': x2,
                   })

# 表の先頭を表示
df.head()

,x1,x2
0,0,0
1,1,1
2,0,1
3,1,1
4,1,0


#### 上記で変数$x_1$と変数$x_2$を生成したのち、その値に応じて、変数$x_3$を生成する .

In [4]:
# 変数x3：0か1の値をnum_data個生成する
# (x1,x2)= (0,0)のとき、0の確率は0.2
# (x1,x2)= (1,0)のとき、0の確率は0.3
# (x1,x2)= (0,1)のとき、0の確率は0.4
# (x1,x2)= (1,1)のとき、0の確率は0.1

x3 = []
for i in range(num_data):
  if x1[i] == 0 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.2, 0.8])
    x3.append(x3_value[0])  # x3はリストになっているので、0番目の要素を取り出して追加
  elif x1[i] == 0 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.3, 0.7])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 0:
    x3_value = np.random.choice([0, 1], 1, p=[0.4, 0.6])
    x3.append(x3_value[0])
  elif x1[i] == 1 and x2[i] == 1:
    x3_value = np.random.choice([0, 1], 1, p=[0.1, 0.9])
    x3.append(x3_value[0])

df["x3"] = x3

# データXの表示
df

,x1,x2,x3
0,0,0,1
1,1,1,1
2,0,1,0
3,1,1,1
4,1,0,1
5,0,1,1
6,0,1,1
7,1,0,1
8,1,1,1
9,1,1,1


#### ・ここで、注意点として、ベイジアンネットワークのモデルmはDAGの形を規定し、変数間の因果の有無、因果の方向性は指定するが、CPT(条件付き確率表)の具体的な確率値、例えば$P(x_1=0)$の値などは規定しない .

#### このCPTの具体的な確率値は得られたデータから計算し、そして計算した確率値のもとでデータの対数尤度を$l_m(\theta_m \mid X)$として求める .

#### ・対数尤度とは、統計モデルが観測データをどの程度適切に説明できるかを測る尺度のこと .

#### ・CPT(条件付き確率表)の具体的な確率値を$\hat{\theta}_{i,j,k}$と表す .　ここで$i$は変数$i$を示す .　$j$は変数$i$の条件のとあるパターンを示す .

#### 例えば今回の変数$x_3$であれば、4つのパターンが存在する .　最後に$k$は変数$i$の値を示す .

#### ・CPT(条件付き確率表)の具体的な確率値$\hat{\theta}_{i,j,k}$は

$$
\hat{\theta}_{i,j,k} = \frac{N_{ijk}}{N_{ij}}
$$

#### として計算される .　ここで、${N_{ijk}}$は変数$i$がとある条件パターン$j$で値$k$となったデータ数である .　${N_{ij}}$は変数$i$がとある条件パターン$j$であったデータ数である .

#### ただし、上記の計算式は、データが多項分布に従い、データの生成確率はすべてハイパーパラメータ0のディリクレ分布を仮定している .

#### ・変数$x_1$から順番に、正解のDAGの形、モデルmを与えて、表7.2.1のデータ$X$に対する$\hat{\theta}_{i,j,k}$を計算してみる .

#### ・変数$x_1$は条件付き確率ではないので、条件パターン$j$は存在しない .　よって$\hat{\theta}_{1,[],0}$は10個のデータから変数$x_1$が0の数を求めると4つであり、$N_{1[]0} = 4$である .



$$
\hat{\theta}_{1,[],0} = \frac{4}{10} = 0.4
$$

#### となる .　$\hat{\theta}_{1,[],0}$はすなわち、$P(x_1=0)$の推定値なので、$\hat{\theta}_{1,[],1}$は$1-\hat{\theta}_{1,[],0}$となり、0.6と求まる .

#### $P(x_1=0)=0.4$と$P(x_1=1)=0.6$が推定値で、真の答えは0.6と0.4だったので、正しい確率値が推定できていない .　これはデータ数が10個と少ないためである .

#### ・同様にして変数$x_2$の$\hat{\theta}_{2,[],0}$を求めると、

$$
\hat{\theta}_{2,[],0} = \frac{3}{10} = 0.3
$$

#### となる .　よって$\hat{\theta}_{2,[],1}$は0.7である .　真の答えの0.4と0.6からは若干ずれているが、まずまずの確率が推定されている .

#### ・最後に変数$x_3$のCPT(条件付き確率)を推定する .　変数$x_3$は変数$x_1$と変数$x_2$から因果を持ち、その条件付き確率で表されるため、条件パターン$j$は$[0,0]$,$[0,1]$,$[1,0]$,$[1,1]$の4つとなる .

#### $\hat{\theta}_{3,[0,0],0}$を求めると、$(x_1,x_2,x_3)=(0,0,0)$のデータが観測されておらず、$(x_1,x_2,x_3)=(0,0,1)$のデータは1つのため、

$$
\hat{\theta}_{3,[0,0],0} = \frac{0}{1} = 0.0
$$

#### となる .　$\hat{\theta}_{3,[0,0],1}$は1.0となる .　正解の0.2と0.8とは少しずれた推定結果である .

#### ・同様に他のパターンも計算する .　条件パターン$j=[0,1]$の場合は

$$
\hat{\theta}_{3,[0,1],0} = \frac{1}{3} = 0.33
$$

#### となり、$\hat{\theta}_{3,[0,1],1}$は0.67である .　$\hat{\theta}_{3,[1,0],0}$は 

$$
\hat{\theta}_{3,[1,0],0} = \frac{0}{2} = 0.0
$$

#### となり、$\hat{\theta}_{3,[1,0],1}$は1.0である .　$\hat{\theta}_{3,[1,1],0}$は

$$
\hat{\theta}_{3,[1,1],0} = \frac{0}{4} = 0.0
$$

#### となり、$\hat{\theta}_{3,[1,1],1}$は1.0となる .

#### ・以上で正解のDAGの構成mのもとでの、表7.2.1のデータ$X$に対する$\hat{\theta}_{i,j,k}$をすべて計算することができた .　最後に推定したCPT(条件付き確率表)を使用してデータ$X$の対数尤度$l_m(\theta_m \mid X)$を求める .　

#### ただし、$l_m(\theta_m \mid X)$そのものの値ではなく、$l_m(\theta_m \mid X)$に比例する値を計算する .　正確な$l_m(\theta_m \mid X)$の値は計算が大変なため、代わりに比例する値を計算する(複数のDAGの妥当性を比較するうえでは比例値を比較することで問題ない) .

#### ・$l_m(\theta_m \mid X)$に比例する値は、

$$
l_m(\theta_m \mid X) \propto \sum_i \sum_j \sum_k (N_{ijk}) \log \hat{\theta}_{i,j,k}
$$

#### で計算される .　表7.2.1の場合は、

$$
l_m(\theta_m \mid X) \propto 4 \times \log 0.4 + 6 \times \log 0.6 + 3 \times \log 0.3 + 7 \times \log 0.7 + 1 \times \log 1.0 + 1 \times \log 0.33 \\
+ 2 \times \log 0.67 + 2 \times \log 1.0 + 4 \times \log 1.0 = -14.7
$$

#### となる .

#### ・最後にBICの計算に戻ると、

$$
BIC_m = -2\, l_m(\theta_m \mid X) + k_m (\log N)
$$

#### であったので、

$$
BIC_m = -2 \times -14.7 + 6 \times (\log 10) = 43.3
$$

#### となります .

#### ・以上により、正解のDAG、すなわちとあるベイジアンネットワークmに対する、計測されたデータのBIC値を計算することができた .

#### 実際には正解のDAGの形mは不明なので、複数のDAGのBIC値を比較することになる .

#### ・ここまで手計算でBICを求めたが、実装時にはベイジアンネットワークのライブラリを活用する .　ベイジアンネットワークを扱えるPythonのライブラリはいくつか種類が存在するが、本書ではpgmpy(Python library for Probabilistic Graphical Models)を利用する .

In [8]:
!pip install pgmpy==0.1.9

#### 正解のDAGを与える .

In [ ]:
# 正解のDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x1', 'x3'), ('x2', 'x3')])  # x1 -> x3 <- x2


ImportError: cannot import name 'config' from 'pgmpy' (/home/taihei/因果分析ゼミ/myenv/lib/python3.13/site-packages/pgmpy/__init__.py)

#### 次に、CPT(条件付き確率表)を計算するベースとなる各パターンでのデータ数を描画、確認する .

In [ ]:
# 各データパターンの個数を表示する
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, df)
print("\n", pe.state_counts('x1'))
print("\n", pe.state_counts('x2'))
print("\n", pe.state_counts('x3'))

#### 次に、CPT(条件付き確率表)を推定する .

In [ ]:
# CPT（条件付き確率表）を推定する
from pgmpy.estimators import BayesianEstimator

estimator = BayesianEstimator(model, df)

cpd_x1 = estimator.estimate_cpd(
    'x1', prior_type="dirichlet", pseudo_counts=[[0], [0]])
cpd_x2 = estimator.estimate_cpd(
    'x2', prior_type="dirichlet", pseudo_counts=[[0], [0]])
cpd_x3 = estimator.estimate_cpd('x3', prior_type="dirichlet", pseudo_counts=[
                                [0, 0, 0, 0], [0, 0, 0, 0]])
# 注意：pseudo_countsはハイパーパラメータ0のディリクレ分布の設定を与えています。

print(cpd_x1)
print(cpd_x2)
print(cpd_x3)

#### 手計算で求めた結果と同じCPTが得られている .　最後にBICを計算する .

In [ ]:
# BICを求める
from pgmpy.estimators import BicScore
bic = BicScore(df)
print(bic.score(model))

#### ・先ほど手計算したBICの値43.3と、pgmpyの出力値が異なっており、43.3ではなく-21.7になっている .　これはpgmpyのBIC計算が、手計算した式

$$
BIC_m = -2\, l_m(\theta_m \mid X) + k_m (\log N)
$$

#### ではなく、

$$
BIC_m = l_m(\theta_m \mid X) - 0.5 \times k_m (\log N) = -14.7 - 0.5 \times 6 \times (\log 10)
$$

#### として計算されているためである .　手計算した式に-0.5が掛け算されている .　BICの計算にもいくつか種類があり、手計算したものは基本的に良く使用されるもの、pgmpyの計算式も使用される .

#### ・最後に正解ではないDAGでのBICを計算してみる .　正解のDAGは変数$x_3$が変数$x_1$と変数$x_2$から因果を持つ構造であったが、変数$x_2$から変数$x_1$,$x_3$に因果関係があるとする .

In [ ]:
# 正解ではないDAGを与える
from pgmpy.models import BayesianModel
model = BayesianModel([('x2', 'x1'), ('x2', 'x3')])  # x1 <- x2 -> x3
bic = BicScore(df)
print(bic.score(model))

#### ・先ほどの正解のDAGでのBICが-21.6であったのに対して、-21.4と大きな値になった .　ここで使用しているpgmpyのBICの定義では、より大きな値になるほど(負の方向に小さくなるほど)良いモデルであるため、今回はたまたま正解ではないDAGの方が値が良くなっている .

#### 原因としてデータ数が少ないことが主であると考えられる .

## 7-3 変数間の独立性の検定

#### ・本節では観測したデータから、2つの変数の間に因果関係が存在しているのか、それとも独立な変数なのかを判定する手法について解説する .

#### ただし、独立性を確かめる手法で、変数間の因果関係の有無は分かるが、因果の方向性は分からない .　また2変数が独立でなかった場合に、直接的な因果関係にあるのか、別の変数を介した間接的な因果関係にあるのかは分からない .

#### ・しかしこの独立性の検定を繰り返して利用することで、どの変数間に因果関係があるのかを明らかにし、DAGを推定するベースとなるスケルトンを求めることができる .

#### ・はじめに独立性の検定の仕組みを解説し、次に疑似データに対して独立性検定の実装、実施を行う .

## 独立性の検定

#### ・変数間の条件付きの独立性の検定について解説する .

#### ・ここで"条件"を$C$と表すことにする .　例えば、変数$x_3=1$などである .　条件$C$が存在しないケースもある .

#### ・次に独立性を検定したい変数を$x_1$,$x_2$とここでは仮定する .

#### すると、2変数間の条件付き独立関数は$CI(x_i,x_j|C)$と表現され、上記の例の場合、$CI(x_1,x_2|x_3=1)$と記載される .　ここで、CIはConditional independence(条件付き独立)を意味する .

#### ・この$CI(x_1,x_2|x_3=1)$の検定として、"独立性のカイ二乗検定"や"G^2テスト"(G-square testもしくはG検定と呼ぶ)が使用される .　本書では独立性のカイ二乗検定について解説する .

## 独立性のカイ二乗検定

#### ・統計検定では帰無仮説と呼ばれる仮説を検討する .　独立性の検定の場合は、帰無仮説は「変数$x_1$と変数$x_2$は独立である」である .　帰無仮説は$H_0$で表記される .

#### ・帰無仮説が棄却されると、「変数$x_1$と変数$x_2$は独立ではない」と判定できる .　帰無仮説が棄却できない場合は、「変数$x_1$と変数$x_2$は独立」と判定できるわけではなく、「変数$x_1$と変数$x_2$は独立なのか独立ではないのかは判断できない」という判定になる .

#### ・ここで、カイ二乗統計量と呼ばれる指標をデータから計算し、カイ二乗統計量の値が「変数$x_1$と変数$x_2$は独立である」と仮定した場合に比べて、著しく大きければ前提である独立という仮定がおかしかったということになる .　すると、最初に立てた帰無仮説が棄却できる .

#### ・表7.3.1のようなデータが得られていたとする .　表7.3.1は独立性の検定をしたいデータ$X$の各条件での頻度(各個数)となっている .　今回は変数$x_1$と変数$x_2$は独立ではなく、因果関係にあるとする .

![alt text](pict5.png)

#### 以下の実装により、表7.3.1のデータを生成している .

In [ ]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [ ]:
# 使用するパッケージ（ライブラリと関数）を定義
from numpy.random import *
import pandas as pd

In [ ]:
# データ数
num_data = 100

# x1：0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2：0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# x2はx1と因果関係にあるとする
x2 = x2*x1

# 2変数で表にする
df = pd.DataFrame({'x1': x1,
                   'x2': x2,
                   })

# 表の先頭を表示
df.head()

In [ ]:
# 各カウント
print(((df["x1"] == 0) & (df["x2"] == 0)).sum())
print(((df["x1"] == 1) & (df["x2"] == 0)).sum())
print(((df["x1"] == 0) & (df["x2"] == 1)).sum())
print(((df["x1"] == 1) & (df["x2"] == 1)).sum())

#### ・表7.3.1の変数$x_1$と変数$x_2$の分布表(度数分布表)からカイ二乗統計量を計算する .　カイ二乗統計量は、

$$
\chi^2 = \sum_{i=1}^r \sum_{j=1}^c \frac{(n_{ij} - E_{ij})^2}{E_{ij}}
$$

#### で計算される .　ここで、$r$は度数分布表の行数、$c$は列数である .　$n_{ij}$は$i$行目$j$列目の度数を示す .　$E_{ij}$は$i$行目$j$列目の推定期待度数である .

#### ・推定期待度数とは、変数$x_1$と変数$x_2$が独立であったとすればどの程度の値になるかを示し、

$$
E_{ij} = \frac{n_i \times n_j}{N}
$$

#### で計算される .　ここで$n_i$は$i$行目のデータの総数、$n_j$は$j$列目のデータの総数、$N$は全データ数である .

#### ・表7.3.2に推定期待度数の表を示す .　これは例えば$E_{11}$の推定期待度数は、

$$
E_{11} = \frac{58\times67}{100}
$$

#### のように計算される .

![alt text](pict6.png)

#### 表7.3.1の観測データの度数と、表7.3.2の推定期待度数より、カイ二乗統計量は

$$
\chi^2 = 
\frac{(58 - 38.9)^2}{38.9}
+ \frac{(0 - 19.1)^2}{19.1}
+ \frac{(9 - 28.1)^2}{28.1}
+ \frac{(33 - 13.9)^2}{13.9}
= 68.0
$$

#### となる .

#### ・次にこのカイ二乗統計量と、変数$x_1$と変数$x_2$が独立である場合の値と比較する .

#### ・カイ二乗統計量は度数分布表の行数、列数で値が大きく変わるので、その点を調整してあげる必要がある .　行数や列数が多いほどカイ二乗統計量の値も大きくなる .

#### その調整のための考慮する量を自由度と呼ぶ .

#### ・自由度は度数分布表の行数$r$、列数$c$を利用して、$(r-1)(c-1)$と計算する .　1を引くのは確率や度数は全体のデータ数が分かっていれば、最後の行や列の値は、その他の行と列から計算(1.0-その他の総和)で計算できるからである .

#### ・最後にこの自由度での変数$x_1$と変数$x_2$は独立である場合のカイ二乗統計量と比較して、今回のデータから得られたカイ二乗統計量が、どれくらい滅多にないことなのかを求める .

#### そのためには、カイ二乗分布表と比べることになる .

![alt text](pict7.png)

#### ・カイ二乗分布表から、自由度1の変数$x_1$と変数$x_2$が独立である場合のカイ二乗統計量は50%の確率で0.455以下と分かる .　同様に90%の確率で2.71以下、95%の確率で3.84以下と分かる .

#### ・どれくらい珍しいデータであれば、それは独立でない変数から生まれたと判定するのかは、有意確率$p$をあらかじめ決めておくことになる .

#### 一般的には$p=0.05$で、5%以下でしか発生しないようなデータが得られた場合は、初めに立てた帰無仮説、すなわち今回の場合は「変数$x_1$と変数$x_2$は独立である」に無理があったと判定して、2つの変数は関連している、因果の関係にあると判断する .

#### ・今回のデータから得られたカイ二乗統計量は68.0であり、自由度1のカイ二乗分布表は95%の確率で3.84以下です .

#### 言い換えると、3.84以上の値を取るのは5%以下の珍しいケースであるということである .

#### 今回$p=0.05$を棄却の基準とした場合、5%以下でしか発生しない状況になっている .　そのため、「変数$x_1$と変数$x_2$は独立である」という考えに無理があった、すなわち帰無仮説を棄却し、「変数$x_1$と変数$x_2$は独立ではない、関連しており、因果関係にある」と判定する .

## 条件付きの独立性のカイ二乗検定

#### ・条件付きの状況においても独立性のカイ二乗検定は同じ手順となる .

#### 独立性を検定したい2変数について、その条件のもとでの度数分布表を作成し、推定期待度数を求め、自由度を計算し、その自由度での変数$x_1$と変数$x_2$は独立である場合のカイ二乗統計量と比較して、帰無仮説が棄却できるかを判定する .

## pgmpyでの実装

#### ・2変数が独立の場合に、7.2節で使用したpgmpyで独立性を検定する .

In [ ]:
# データ数
num_data = 100

# x1：0か1の値をnum_data個生成、0の確率は0.6、1の確率は0.4
x1 = np.random.choice([0, 1], num_data, p=[0.6, 0.4])

# x2：0か1の値をnum_data個生成、0の確率は0.4、1の確率は0.6
x2 = np.random.choice([0, 1], num_data, p=[0.4, 0.6])

# 2変数で表にする
df2 = pd.DataFrame({'x1': x1,
                   'x2': x2,
                   })

# 各カウント
print(((df2["x1"] == 0) & (df2["x2"] == 0)).sum())
print(((df2["x1"] == 1) & (df2["x2"] == 0)).sum())
print(((df2["x1"] == 0) & (df2["x2"] == 1)).sum())
print(((df2["x1"] == 1) & (df2["x2"] == 1)).sum())

#### ・独立性の検定を実施する .　なお本節の最初で使用したデータ(因果関係があり、独立でないバージョン)にも、検定を実施する .

In [ ]:
from pgmpy.estimators import ConstraintBasedEstimator

est = ConstraintBasedEstimator(df2)
print(est.test_conditional_independence(
    'x1', 'x2', method="chi_square", tol=0.05))  # 独立

# 最初の例の場合
est = ConstraintBasedEstimator(df)
print(est.test_conditional_independence(
    'x1', 'x2', method="chi_square", tol=0.05))   # 独立でない

#### ・出力はTrue,Falseとなり、今回新たに独立なデータを作成したケースでは独立と判定され、本節の最初から使用してきた独立でないデータでは、Falseとして独立とは言えないと正しく判定された .

## 7-4　3タイプのベイジアンネットワークの探索手法

#### ・本節では観測したデータから、ベイジアンネットワークのDAGを推定する手法について解説する .

#### ・3タイプのネットワーク探索手法とは、以下の通りである .

#### ①スコアリングによる構造学習(Score-based Structure Learning)
#### ②条件付き独立性検定による構造学習(Constraint-based Structure Learning)
#### ③ベイジアンスコアと条件付き独立性検定のハイブリッド型構造学習(Hybrid Structure Learning)

#### ・①スコアリングによる構造学習とは、7.2節で解説した、観測データに対するネットワークの当てはまりの良さを示すAIC,BIC,MDL符号,BDe,BDeu,K2などのベイジアンスコアのいずれかを使用し、様々なDAGに対してスコアを求め、最も指標値の良いDAGを選ぶ手法のこと .

#### スコアリングによる構造学習は、手法がシンプルという利点がある一方で、